<a href="https://colab.research.google.com/github/ubermenchh/ArticleChat/blob/main/ArticleChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q llama-index langchain pypdf sentence-transformers sentencepiece arxiv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 917.6/917.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 8.3 MB/s

In [2]:
from google.colab import userdata
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('hf_read_token')

In [5]:
from llama_index import SummaryIndex
from llama_index.readers import TrafilaturaWebReader
from IPython.display import Markdown, display
import os

In [6]:
documents = TrafilaturaWebReader().load_data(
    ['http://karpathy.github.io/2019/04/25/recipe/']
)

In [7]:
len(documents)

1

In [9]:
import openai
openai.api_key = None

In [11]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from langchain.llms import HuggingFaceHub
from llama_index.llms import LangChainLLM

llm = LangChainLLM(llm=HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-beta', model_kwargs={'temperature': 0.2, 'max_tokens': 4096, 'top_p': 0.95}))
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/134M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
query_engine = index.as_query_engine()
response = query_engine.query("What does becoming one with the data mean?")

In [13]:
display(Markdown(f"<b>{response}</b>"))

<b>

Becoming one with the data refers to thoroughly inspecting and understanding the distribution and patterns of the data before training a neural network. This step is critical as it helps in discovering issues with the data, such as duplicate examples, corrupted images, or label imbalances. It also provides insights into the types of architectures that will be explored and helps in understanding the network's predictions. By gaining a qualitative sense of the data, it becomes easier to identify where the network might be going wrong and to write simple code to search, filter, and sort the data to visualize its distribution and outliers. This step is essential in building trust in the correctness of the training and evaluation skeleton and in preventing the introduction of bugs or misconfigurations that will take forever to find.</b>

In [33]:
def load_url(url):
    documents = TrafilaturaWebReader().load_data([url])
    llm = LangChainLLM(llm=HuggingFaceHub(repo_id='HuggingFaceH4/zephyr-7b-beta', model_kwargs={'temperature': 0.2, 'max_tokens': 4096, 'top_p': 0.95}))
    service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    return index

In [34]:
index = load_url('https://pytorch.org/blog/accelerating-generative-ai-2/')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [35]:
index

In [36]:
def chat(index, query='Summarize the article.'):
    query_engine = index.as_query_engine()
    response = query_engine.query(query)
    return display(Markdown(f"<b>{response}</b>"))

In [37]:
chat(index)

<b>

The article discusses techniques to improve the performance of text generation using PyTorch and NVIDIA GPUs. The first technique is using torch.jit to compile the model, which reduces the overhead of dynamic computation and improves performance by 1.5x. The second technique is int8 weight-only quantization, which reduces the size of weights and improves performance by 50%. The third technique is speculative decoding, which breaks the strict serial dependency in autoregressive generation and improves performance by more than 4x. The article also explains how torch.compile helps to improve performance by reducing overhead and generating faster kernels. The article concludes by measuring the Model Bandwidth Utilization (MBU) to understand the theoretical limit of performance and how close the techniques are to it.</b>

In [38]:
chat(index, query='What is the title of this article?')

<b> "Fast Inference for Large Language Models with PyTorch"</b>